## **USER ACTION REQUIRED**

- Upload ```silero_vad.onnx```
- Upload ```waves_snr.txt```

In [1]:
num_test_examples = 80

## **Define ASR Model**

In [2]:
!nvcc --version

nvcc: NVIDIA (R) Cuda compiler driver
Copyright (c) 2005-2024 NVIDIA Corporation
Built on Thu_Jun__6_02:18:23_PDT_2024
Cuda compilation tools, release 12.5, V12.5.82
Build cuda_12.5.r12.5/compiler.34385749_0


In [3]:
!pip uninstall -y torch torchvision torchaudio triton nvidia-cublas-cu12 nvidia-cuda-runtime-cu12 nvidia-cudnn-cu12 nvidia-cufft-cu12 nvidia-curand-cu12 nvidia-cusolver-cu12 nvidia-cusparse-cu12 nvidia-nccl-cu12

Found existing installation: torch 2.6.0+cu124
Uninstalling torch-2.6.0+cu124:
  Successfully uninstalled torch-2.6.0+cu124
Found existing installation: torchvision 0.21.0+cu124
Uninstalling torchvision-0.21.0+cu124:
  Successfully uninstalled torchvision-0.21.0+cu124
Found existing installation: torchaudio 2.6.0+cu124
Uninstalling torchaudio-2.6.0+cu124:
  Successfully uninstalled torchaudio-2.6.0+cu124
Found existing installation: triton 3.2.0
Uninstalling triton-3.2.0:
  Successfully uninstalled triton-3.2.0
Found existing installation: nvidia-cublas-cu12 12.5.3.2
Uninstalling nvidia-cublas-cu12-12.5.3.2:
  Successfully uninstalled nvidia-cublas-cu12-12.5.3.2
Found existing installation: nvidia-cuda-runtime-cu12 12.5.82
Uninstalling nvidia-cuda-runtime-cu12-12.5.82:
  Successfully uninstalled nvidia-cuda-runtime-cu12-12.5.82
Found existing installation: nvidia-cudnn-cu12 9.3.0.75
Uninstalling nvidia-cudnn-cu12-9.3.0.75:
  Successfully uninstalled nvidia-cudnn-cu12-9.3.0.75
Found exi

In [4]:
!pip install torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu125

Looking in indexes: https://pypi.org/simple, https://download.pytorch.org/whl/cu125
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.7/766.7 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 112.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 87.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 52.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 10.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 39.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 188.7/188.7 MB 11.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

In [5]:
!pip install -q datasets bitsandbytes accelerate onnxruntime evaluate jiwer

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 487.4/487.4 kB 31.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 76.1/76.1 MB 30.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.0/16.0 MB 106.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.0/84.0 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 11.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 183.9/183.9 kB 15.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 143.5/143.5 kB 13.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 90.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 kB 4.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.8/194.8 kB 19.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.8/86.8 kB 8.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is 

In [6]:
import torch
import triton
import torch.backends.cudnn as cudnn

print("Torch Version:", torch.__version__)
print("Torch CUDA Version:", torch.version.cuda)
print("CUDA Available:", torch.cuda.is_available())
print("Triton Version:", triton.__version__)
print("cuDNN Version:", cudnn.version())

Torch Version: 2.6.0+cu124
Torch CUDA Version: 12.4
CUDA Available: True
Triton Version: 3.2.0
cuDNN Version: 90100


In [7]:
from transformers import WhisperForConditionalGeneration, WhisperProcessor
from google.colab import drive
import time
import warnings
import numpy as np
import onnxruntime
from datasets import load_dataset
from itertools import islice
from torch.utils.data import IterableDataset
import json
import os
import wave
import threading
import textwrap
from IPython.display import clear_output
import evaluate
from transformers.models.whisper.english_normalizer import BasicTextNormalizer
import collections
import pickle
import random

<br/>
<br/>
<br/>

**User Action Required**
- Define model to use

In [8]:
# Google Drive Env Setup
whisper_ver = 'whisper-base'
checkpoint_num = '5400'
drive.mount('/content/drive')
checkpoint_path = f'/content/drive/My Drive/{whisper_ver}-noiseaugmented-continued-checkpoints/checkpoint-{checkpoint_num}'

# Model setup code for fine-tuned whisper
device = "cuda:0" if torch.cuda.is_available() else "cpu"
model = WhisperForConditionalGeneration.from_pretrained(checkpoint_path).to(device)
model.config.use_cache = True
processor = WhisperProcessor.from_pretrained(f"openai/{whisper_ver}", language="en", task="transcribe")

Mounted at /content/drive


preprocessor_config.json:   0%|          | 0.00/185k [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/283k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/836k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.48M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/494k [00:00<?, ?B/s]

normalizer.json:   0%|          | 0.00/52.7k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/34.6k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/2.19k [00:00<?, ?B/s]

## **Load HF Datasets**

**User Action Required**

- Define the number of test samples

In [9]:
dataset_repo_test = "johnlohjy/imda_nsc_p3_test_noiseaugmented"
dataset_test = load_dataset(dataset_repo_test, split='test', streaming=True, trust_remote_code=True)

imda_nsc_p3_test_noiseaugmented.py:   0%|          | 0.00/3.96k [00:00<?, ?B/s]

In [10]:
class SlicedDataset(IterableDataset):
    def __init__(self, dataset, num_examples):
        self.dataset = dataset
        self.num_examples = num_examples

    def __iter__(self):
        return islice(iter(self.dataset), self.num_examples)

    def __len__(self):
        return self.num_examples

dataset_test_reduced = SlicedDataset(dataset_test, num_examples=num_test_examples)

<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>
<br/>

## **Transcribe chunks of 25s, 20s, 15s, 10s**

In [11]:
samples_25s = []
samples_20s = []
samples_15s = []
samples_10s = []

sample_rate = 16000

for sample in dataset_test_reduced:
  # Get a single audio samples data and convert from float 64 to float 32
  audio_sample = sample["audio"]["array"].astype(np.float32)

  # Compute samples for each duration and append it
  samples_25s.append(audio_sample[:25 * sample_rate])
  samples_20s.append(audio_sample[:20 * sample_rate])
  samples_15s.append(audio_sample[:15 * sample_rate])
  samples_10s.append(audio_sample[:10 * sample_rate])

  print('')
  print(sample)


{'path': 'waves/3000-1_11.wav', 'audio': {'path': 'waves/3000-1_11.wav', 'array': array([ 0.00030518,  0.00064087,  0.00042725, ..., -0.02471924,
       -0.01220703,  0.00460815]), 'sampling_rate': 16000}, 'sentence': "you know yeah she was from france so that's she shared with me about her culture lah so you know err she would show me around when she would when i would visit lah when i travel to france looking forward to that it's really not interesting generation gap what's the biggest similarity or difference between your generation and your parent's generation okay"}

{'path': 'waves/3000-1_17.wav', 'audio': {'path': 'waves/3000-1_17.wav', 'array': array([0.00015259, 0.00039673, 0.0007019 , ..., 0.00112915, 0.00088501,
       0.00439453]), 'sampling_rate': 16000}, 'sentence': "would you say it's more positive or negative neutral lah good for the country in the in having as in globalization is good for the country okay okay um never yeah okay next tikam tikam how do you choose four

In [16]:
def transcribe_audio(input_bytes):
    input_features = processor.feature_extractor(input_bytes, sampling_rate=16000).input_features[0]
    input_features = torch.tensor(input_features).unsqueeze(0).to(device)
    generated_ids = model.generate(input_features)
    last_segment = processor.tokenizer.decode(generated_ids[0], skip_special_tokens=True)
    return last_segment

In [17]:
samples_25s_transcribe_time = []
samples_20s_transcribe_time = []
samples_15s_transcribe_time = []
samples_10s_transcribe_time = []

for i in range(len(samples_25s)):
  start_time = time.time()
  res = transcribe_audio(samples_25s[i])
  end_time = time.time()
  print('')
  print(res)
  samples_25s_transcribe_time.append(end_time - start_time)


you know ya she was from france and it's to share with me about her culture so you know uh she would she want to around and she would when i would visit lah when i travel her house looking forward to that it's for them what interesting thing generation gap what's the biggest similarity or difference between your generation and your

would you say it's more positive or negative neutral lah good for the country in the in having as in globalization is good for the country okay okay never ya okay next take come take come how do you choose four day or total numbers mm okay

what most busybody thing i've ever done i i don't really know like it's it's like small events like like what's most busybody thing i see

there was this guy who just came out of nowhere when we were in the cemetery you know he told us that his name was um yan wei so i thought it was a little bit weird yan wei so i kept i kept repeating the name to myself yan wei and i found it really familiar when i got back home

weir